In [ ]:
!pip install pygame
!pip install pygad

!git clone https://github.com/mkaanaslan/evoman.git tmp
!cp -r /kaggle/working/tmp/* /kaggle/working/
!rm -R /kaggle/working/tmp

In [ ]:
import sys, os
import numpy as np
 
import random
from evoman.environment import Environment
from evoman.controller import Controller
from pygad.kerasga import model_weights_as_vector,model_weights_as_matrix

import tensorflow as tf 
tf.compat.v1.disable_eager_execution()

from deap import base, creator, tools, algorithms


In [ ]:
class player_controller(Controller):
    
    def __init__(self, model):
        self.model = model

    def control(self, inputs, controller):

        # Normalises the input using min-max scaling
        inputs = (inputs-min(inputs))/float((max(inputs)-min(inputs)))
        inputs = np.array(inputs).reshape((1,20,))

        sol = model_weights_as_matrix(self.model,controller)
        self.model.set_weights(sol)
        preds = self.model.predict(inputs,verbose=0)
        preds = (preds>0.5).astype(int)

        return preds[0]

In [ ]:
def fitness_function(solution):
    return (env.play(solution)[0],)

In [ ]:
input_layer  = tf.keras.layers.Input((20,))
#dense_layer1 = tf.keras.layers.Dense(16, activation="relu")(input_layer)
output_layer = tf.keras.layers.Dense(5, activation="sigmoid")(input_layer)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

In [ ]:
headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"

env = Environment(playermode="ai",
                  player_controller=player_controller(model),
                  speed="fastest",
                  enemymode="static",
                  enemies="1",
                  level=2,
                  visuals=False)

In [ ]:
individual_size = model.count_params()

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [ ]:
toolbox = base.Toolbox()
toolbox.register("weight_bin", np.random.uniform,-1,1)   #Initiate weights from uniform distribution between -1 to +1
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.weight_bin, n=individual_size)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


In [ ]:
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.01)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", fitness_function)

In [ ]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("Mean", np.mean)
stats.register("Max", np.max)
stats.register("Min", np.min)

In [ ]:
pop = toolbox.population(n=100)
hof = tools.HallOfFame(1)

In [ ]:
%%time
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.8, mutpb=0.2, ngen=10, halloffame=hof, stats=stats,verbose=True)